In [1]:
!pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 3.0 MB/s eta 0:00:00


In [2]:
import patoolib

In [3]:
patoolib.extract_archive("hrdataset.zip")

INFO patool: Extracting hrdataset.zip ...
INFO:patool:Extracting hrdataset.zip ...
INFO patool: running /usr/bin/7z x -aou -o./Unpack_kvdr5uux -- hrdataset.zip
INFO:patool:running /usr/bin/7z x -aou -o./Unpack_kvdr5uux -- hrdataset.zip
INFO patool: ... hrdataset.zip extracted to `hrdataset'.
INFO:patool:... hrdataset.zip extracted to `hrdataset'.


'hrdataset'

In [ ]:
/content/hrdataset/employees

In [6]:
import os
import pandas as pd

# Directory containing markdown files
markdown_dir = "/content/hrdataset/employees"

# Parse markdown files into a DataFrame
employee_data = []
for filename in os.listdir(markdown_dir):
    if filename.endswith(".md"):
        with open(os.path.join(markdown_dir, filename), "r") as f:
            lines = f.readlines()
            profile = {}
            for line in lines:
                if line.startswith("- **Name:**"):
                    profile["Name"] = line.split(":")[1].strip("**").strip("\n").strip(" ")
                elif line.startswith("- **Role:**"):
                    profile["Role"] = line.split(":")[1].strip("**").strip("\n").strip(" ")
                elif line.startswith("- **Department:**"):
                    profile["Department"] = line.split(":")[1].strip("**").strip("\n").strip(" ")
                elif line.startswith("- **Joining Date:**"):
                    profile["Joining Date"] = line.split(":")[1].strip("**").strip("\n").strip(" ")
            if profile:
                employee_data.append(profile)

# Convert to DataFrame
employee_df = pd.DataFrame(employee_data)
print(employee_df)

              Name                Role       Department Joining Date
0     Sunita Patil   Finance Executive          Finance   2022-01-17
1    Neha Malhotra      Junior Analyst        Logistics   2023-07-01
2       Anjali Das        HR Executive  Human Resources   2021-05-10
3       Amit Verma                 CEO        Executive   2016-02-01
4      Aditya Jain    Senior Developer               IT   2019-06-10
5      Rohit Mehra   Logistics Analyst        Logistics   2020-08-22
6  Rajesh Kulkarni                 CTO        Executive   2017-11-15
7     Priya Sharma  Operations Manager       Operations   2019-03-15
8     Karan Kapoor    Fleet Supervisor        Logistics   2018-11-03
9       Meera Iyer   Marketing Manager        Marketing   2020-02-20


# Summarization Models
These models are pre-trained for text summarization tasks.

* facebook/bart-large-cnn: The go-to model for
summarization, trained on large news datasets.

* t5-small, t5-base, t5-large: Highly versatile models that can perform summarization and many other tasks.
  **Usage:** Prefix input with "summarize: ".

* pegasus-xsum (Google Research): Specialized in abstractive summarization.
* google/long-t5-tglobal-base: Designed for summarizing longer documents effectively.

In [7]:
from transformers import pipeline

# Load pre-trained summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Combine employee details for a specific department
operations_data = " ".join([
    f"{row['Name']} is the {row['Role']} in {row['Department']}, joined on {row['Joining Date']}."
    for _, row in employee_df.iterrows() if row['Department'] == "Executive"
])

# Summarize the data
summary = summarizer(operations_data, max_length=50,min_length=10, do_sample=False)
print("Summary:", summary[0]['summary_text'])

Device set to use cuda:0
Your max_length is set to 50, but your input_length is only 40. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


Summary: Amit Verma is the CEO in Executive. Rajesh Kulkarni is the CTO in Executive, joined on 2017-11-15.


# Question Answering Models
These models are trained to answer questions based on a provided context.

* distilbert-base-uncased-distilled-squad: A lightweight QA model based on the SQuAD dataset.

* bert-large-uncased-whole-word-masking-finetuned-squad: A more powerful BERT-based QA model.

* deepset/roberta-base-squad2: Trained on SQuAD2.0, supports unanswerable questions.

In [11]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
# context = "Priya Sharma is the Operations Manager who joined the company in 2019."
context = summary[0]['summary_text']
question = "Who is the CEO?"
answer = qa_pipeline(question=question, context=context)
print(answer)

Device set to use cuda:0


{'score': 0.9947599172592163, 'start': 0, 'end': 10, 'answer': 'Amit Verma'}


# Text Classification Models
For sentiment analysis, topic classification, and other classification tasks.

* distilbert-base-uncased-finetuned-sst-2-english: Sentiment analysis model.
* cardiffnlp/twitter-roberta-base-sentiment: Specifically designed for sentiment analysis on tweets.
* facebook/bart-large-mnli: Can be used for zero-shot text classification.

In [12]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
result = classifier("The employee feedback has been overwhelmingly positive!")
print(result)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9997122883796692}]


# Embedding Models
For similarity search, clustering, and other embedding-based tasks.

* sentence-transformers/all-MiniLM-L6-v2: A lightweight embedding model for generating sentence embeddings.
* openai/clip-vit-base-patch32: Generates embeddings for both text and images.
* bert-base-uncased: You can extract hidden states to use as embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(["This is a test sentence.", "Another example."])
print(embeddings)

# Text Generation Models
For generating text, completing sentences, or creating creative outputs.

* gpt2, distilgpt2: Lightweight generative models for text generation.
* EleutherAI/gpt-neo-2.7B: A larger open-source alternative to GPT-3.
* bigscience/bloom: A multilingual generative model.
* facebook/opt-13b: Efficient, open-source large language model.

In [14]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")
text = generator("Once upon a time, there was a curious employee who", max_length=50)
print(text[0]['generated_text'])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a curious employee who wished to share this news with his friends and acquaintances. He made sure his name was only acknowledged to every living person in this town where he attended college to which he had not paid taxes and no


# Named Entity Recognition (NER) Models
For extracting entities like names, dates, organizations, etc., from text.

* dbmdz/bert-large-cased-finetuned-conll03-english: Fine-tuned on CoNLL-2003 for NER.
* dslim/bert-base-NER: Lightweight NER model

In [20]:
from transformers import pipeline

ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)
text = "Neependra Khare founded CloudYuga in December 2015"
entities = ner(text)
print(entities)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity_group': 'PER', 'score': 0.9995653, 'word': 'Neependra Khare', 'start': 0, 'end': 15}, {'entity_group': 'ORG', 'score': 0.97774523, 'word': 'CloudYuga', 'start': 24, 'end': 33}]


# Translation Models
For translating text between languages.

- Helsinki-NLP/opus-mt-en-de: English-to-German translation.
- t5-small, t5-large: Can also be used for translation by prefixing the input with translate English to French:.

In [21]:
from transformers import pipeline

translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
translation = translator("The employee feedback has been overwhelmingly positive!")
print(translation[0]['translation_text'])

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


Das Feedback der Mitarbeiter war überwältigend positiv!
